In [1]:
%load_ext autoreload
%autoreload 2

import keras
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from Jlayers import SimpleLayer
from utilsSimpleConv2D import*
from SpectralLayer import Spectral
from spectralconvolutions import *
from tensorflow.keras.layers import Dense,Layer
from typing import Tuple,List,Any,Dict
from tensorflow.python.keras import activations, initializers, regularizers, constraints

In [9]:
## Data

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train, x_test=x_train.reshape(-1,28,28,1), x_test.reshape(-1,28,28,1)

x_train, x_test = x_train / 255.0, x_test / 255.0

flat_train = np.reshape(x_train, [x_train.shape[0], 28 * 28])
flat_test = np.reshape(x_test, [x_test.shape[0], 28 * 28])

In [10]:
## parameters

In [3]:
spectral_config={
    'is_base_trainable' : False,
    'is_diag_start_trainable' : False,
    'is_diag_end_trainable' : True,
}

In [5]:
parameters_one={ "use_lambda_out":False,
                 "use_lambda_in":False,
                 "trainable_SM_kernel":True,
                 "use_bias":False,
                 "kernel_initializer":"glorot_uniform"}

In [12]:
## Linear Conv2D: Compact Matrix Ac

In [8]:
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Input(shape=(28,28,1)))


model1.add(SpectralConv2D_one(filters=1, kernel_size=5, strides=1, padding="VALID", activation=None,**parameters_one))
model1.add(keras.layers.MaxPooling2D((2,2)))
model1.add(keras.layers.Dropout(0.2))       

model1.add(SpectralConv2D_one(filters=1, kernel_size=3, strides=1, padding="VALID", activation=None,**parameters_one))
model1.add(keras.layers.MaxPooling2D((2,2)))
model1.add(keras.layers.Dropout(0.2))  
                   
model1.add(keras.layers.Flatten())     

model1.add(Spectral(1000, **spectral_config,use_bias=True, activation='relu'))
model1.add(keras.layers.Dropout(0.5))
model1.add(Spectral(10, **spectral_config,use_bias=False, activation='softmax'))
           
opt = tf.keras.optimizers.Adam(learning_rate=0.03)
model1.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spectral_conv2d_one_2 (Spe  (None, 24, 24, 1)         1385      
 ctralConv2D_one)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 12, 12, 1)         0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 12, 12, 1)         0         
                                                                 
 spectral_conv2d_one_3 (Spe  (None, 10, 10, 1)         253       
 ctralConv2D_one)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 1)           0         
 g2D)                                                 